In [6]:
import os
os.listdir()

['.ipynb_checkpoints',
 'dog-breed-identification.zip',
 'dog_breed_data',
 'subset',
 'Untitled.ipynb']

In [3]:
import zipfile

with zipfile.ZipFile("dog-breed-identification.zip", "r") as zip_ref:
    zip_ref.extractall("dog_breed_data")


In [4]:
os.listdir("dog_breed_data")


['labels.csv', 'sample_submission.csv', 'test', 'train']

In [5]:
import os
import shutil
import sys
import pandas as pd


In [6]:
dataset_dir = 'dog_breed_data/train'
labels = pd.read_csv('dog_breed_data/labels.csv')


In [7]:
import os

def make_dir(x):
    if os.path.exists(x) == False:
        os.makedirs(x)


In [8]:
base_dir = './subset'
make_dir(base_dir)


In [9]:
train_dir = os.path.join(base_dir, 'train')
make_dir(train_dir)


In [11]:
breeds = labels.breed.unique()

for breed in breeds:
    # Make folder for each breed
    breed_dir = os.path.join(train_dir, breed)
    make_dir(breed_dir)

    # Copy images to the corresponding folders
    images = labels[labels.breed == breed]['id']
    for image in images:
        source = os.path.join(dataset_dir, f'{image}.jpg')
        destination = os.path.join(train_dir, breed, f'{image}.jpg')
        shutil.copyfile(source, destination)


In [13]:
os.listdir('subset/train')


['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'greater_swiss_mountain_dog',
 'great_dane',
 'great_pyrenees',
 'groenendael',


In [4]:
dataset = "subset/train"


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
datagen = ImageDataGenerator()
generator = datagen.flow_from_directory(
    dataset,
    target_size=(224, 224),     # Image resized to 224x224
    batch_size=32,              # 32 images per batch
    class_mode='categorical',   # One-hot encoding
    shuffle=False               # IMPORTANT: keeps class order fixed
)


Found 10222 images belonging to 120 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations


In [3]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [4]:
Image_size = [224, 224]

In [5]:
sol = VGG19(
    input_shape=Image_size + [3],
    weights='imagenet',
    include_top=False
)

In [11]:
for i in sol.layers:
    i.trainable = False

In [12]:
y = Flatten()(sol.output)

In [13]:
final = Dense(20, activation='softmax')(y)


In [14]:
from tensorflow.keras.models import Model

vgg19_model = Model(inputs=sol.input, outputs=final)


In [15]:
vgg19_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
vgg19_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['Accuracy']
)


In [17]:
selected_classes = [
    'affenpinscher',
    'beagle',
    'appenzeller',
    'basset',
    'bluetick',
    'boxer',
    'cairn',
    'doberman',
    'german_shepherd',
    'golden_retriever',
    'kelpie',
    'komondor',
    'leonberg',
    'mexican_hairless',
    'pug',
    'redbone',
    'shih-tzu',
    'toy_poodle',
    'vizsla',
    'whippet'
]


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

generator = datagen.flow_from_directory(
    'subset/train',            # path where ONLY 20 folders exist
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=selected_classes,  # 🔥 THIS FIXES YOUR ERROR
    shuffle=True
)


Found 1683 images belonging to 20 classes.


In [19]:
vgg19_model.fit(generator, epochs=6)


Epoch 1/6
53/53 [==============================] - 773s 15s/step - loss: 3.2538 - Accuracy: 0.2074
Epoch 2/6
53/53 [==============================] - 418s 8s/step - loss: 0.9599 - Accuracy: 0.7142
Epoch 3/6
53/53 [==============================] - 837s 16s/step - loss: 0.4265 - Accuracy: 0.9186
Epoch 4/6
53/53 [==============================] - 517s 10s/step - loss: 0.2147 - Accuracy: 0.9816
Epoch 5/6
53/53 [==============================] - 641s 12s/step - loss: 0.1160 - Accuracy: 0.9923
Epoch 6/6
53/53 [==============================] - 585s 11s/step - loss: 0.0626 - Accuracy: 1.0000


In [20]:
vgg19_model.save("dogbreed.h5")


In [36]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import load_model

# Load trained model
vgg19_model = load_model("dogbreed.h5")

# EXACT image path (file, not folder)
img_path = r"C:\Users\patak\Desktop\DOG_BREED_PREDICTION\subset\train\beagle\0566a60d7504a6fad4161d0ef2765a34.jpg"

# Load and preprocess image
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Predict
preds = vgg19_model.predict(x)

class_names = [
    'affenpinscher','beagle','appenzeller','basset','bluetick',
    'boxer','cairn','doberman','german_shepherd','golden_retriever',
    'kelpie','komondor','leonberg','mexican_hairless','pug',
    'redbone','shih-tzu','toy_poodle','vizsla','whippet'
]

print("Predicted Breed:", class_names[np.argmax(preds)])
print("Confidence:", round(np.max(preds) * 100, 2), "%")


1/1 [==============================] - 1s 711ms/step
Predicted Breed: beagle
Confidence: 100.0 %
